<a href="https://colab.research.google.com/github/modaleducation/modaled/blob/main/modaledtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler


> This is a test that I'll be running on a secondary dataset to ensure that the logic behind the ML model is accurate.


In [14]:
cols = ["fLength", "fWidth", "fSize", "fConc", "fConc1", "fAsym	", "fM3Long", "fM3Trans", "fAlpha", "fDist", "class"]
df = pd.read_csv("magic04.data", names=cols)
df.head()


,fLength,fWidth,fSize,fConc,fConc1,fAsym\t,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [15]:
df["class"] = (df["class"] == "g").astype(int)

In [16]:
df.head()

,fLength,fWidth,fSize,fConc,fConc1,fAsym\t,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,1
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,1
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,1
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,1
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,1


In [ ]:
for label in cols[:-1]:
  plt.hist(df[df["class"] == 1][label], color='blue', label='gamma', alpha=0.7, density=True)
  plt.hist(df[df["class"] == 0][label], color='red', label='hadron', alpha=0.7, density=True)
  plt.title(label)
  plt.ylabel("Probability")
  plt.xlabel(label)
  plt.legend()
  plt.show()

In [42]:
train, validation, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])

In [43]:
def scale_dataset(dataframe, oversample=False):
  x = dataframe[dataframe.columns[:-1]].values
  y = dataframe[dataframe.columns[-1]].values

  scaler = StandardScaler()
  x = scaler.fit_transform(x)


  if oversample:
    ros = RandomOverSampler()
    x, y = ros.fit_resample(x, y) # take more of the smaller class and keep sampling from there to increase size of dataset of smaller class
  #hstack is to stack horizontally instead of on top of each other
  #-1 is length of y
  data = np.hstack((x, np.reshape(y, (-1, 1))))

  return data, x, y

In [44]:
train, x_train, y_train = scale_dataset(train, oversample=True)
validation, x_validation, y_validation = scale_dataset(validation, oversample=False)
test, x_test, y_test = scale_dataset(test, oversample=False)

#kNN

In [45]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [46]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(x_train, y_train)

KNeighborsClassifier()

In [47]:
y_predictions = knn_model.predict(x_test)

In [48]:
print(classification_report(y_test, y_predictions))

              precision    recall  f1-score   support

           0       0.75      0.74      0.75      1331
           1       0.86      0.87      0.87      2473

    accuracy                           0.82      3804
   macro avg       0.81      0.80      0.81      3804
weighted avg       0.82      0.82      0.82      3804



#Naive Bayes

In [50]:
from sklearn.naive_bayes import GaussianNB

In [53]:
nb_model = GaussianNB()
nb_model = nb_model.fit(x_train, y_train)

In [54]:
y_predictions = nb_model.predict(x_test)
print(classification_report(y_test, y_predictions))

              precision    recall  f1-score   support

           0       0.66      0.41      0.50      1331
           1       0.73      0.89      0.80      2473

    accuracy                           0.72      3804
   macro avg       0.70      0.65      0.65      3804
weighted avg       0.71      0.72      0.70      3804

